### Imports & Paths (no data is downloaded here)

In [2]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm

DATA_ROOT = Path("data/DIV2K")
HR_VALID = DATA_ROOT / "HR" / "valid"
HR_TRAIN = DATA_ROOT / "HR" / "train"

In [3]:
def lr_dir(scale: int, split: str) -> Path:
    return DATA_ROOT / f"LR_bicubic/X{scale}" / split

Sanity check: do we see the HR images?

In [6]:
def count_images(p: Path):
    return len([x for x in p.glob("*") if x.suffix.lower() in [".png", ".jpg", ".jpeg"]])

print("HR train dir:", HR_TRAIN.resolve())
print("HR valid dir:", HR_VALID.resolve())
print("HR train count:", count_images(HR_TRAIN))
print("HR valid count:", count_images(HR_VALID))

HR train dir: C:\Users\Caleb\Documents\GitHub Repos\PixelForge\notebooks\data\DIV2K\HR\train
HR valid dir: C:\Users\Caleb\Documents\GitHub Repos\PixelForge\notebooks\data\DIV2K\HR\valid
HR train count: 0
HR valid count: 0
